In [11]:
import tensorflow as tf
import cv2
import tensorflow_hub as hub
import requests
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from flask import Flask, jsonify
import json

In [3]:
# Path ke folder dataset
dataset_folder = 'images/'

# Path ke file styles.csv
styles_file = 'styles.csv'

# Membaca data dari file styles.csv
data = pd.read_csv(styles_file)

# Membaca gambar dan label dari folder dataset
images = []

# Memuat label ke dalam variabel
label_csv = data['articleType']

# Mengonversi label FashionImage menjadi tipe data string
label_csv = label_csv.astype(str)

# Menggabungkan gambar dari folder 'images' dengan dataset MNIST
image_folder = 'images/'

In [8]:
model = tf.keras.models.load_model(
    "image_classification_model.h5",
    custom_objects={"KerasLayer": hub.KerasLayer})
label_encoder = LabelEncoder()
label_encoder.fit_transform(label_csv)

array([109, 109, 109, ..., 126, 126, 126])

In [35]:
# Create a Flask application
# app = Flask(__name__)

In [23]:
# @app.route('/predict', methods=['POST'])
def predict(image_url):
    r = requests.get(image_url)
    content = r.content
    
    filename = 'image.jpg'
    base_path = 'uploaded/'
    base_name = os.path.splitext(filename)[0]
    ext = os.path.splitext(filename)[1]
    new_name = base_name + ext
    
    # Buat increment jika file sudah ada di folder '/uploaded'
    i = 1
    while os.path.exists(base_path + new_name):
        new_name = f"{base_name}_{i}{ext}"
        i += 1
    
    full_path = base_path + new_name
    
    with open(full_path, 'wb') as f:
        f.write(content)
        
#     with open('uploaded/image.jpg', 'wb') as f:
#         f.write(r.content)

    ## Membaca gambar
    image = cv2.imread("uploaded/image.jpg")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (28, 28))
    image = np.expand_dims(image, axis=-1)
    image = image.astype('float32') / 255

    ## Melakukan prediksi
    predictions = model.predict(np.array([image]))
    predicted_class = np.argmax(predictions)

    ## Mengonversi kelas prediksi menjadi label
    predicted_label = label_encoder.classes_[predicted_class]

    # Menampilkan hasil prediksi
    print('Predicted Label:', predicted_label)
    
    # Create a dictionary with the prediction
    result = {
        'predicted_label': predicted_label
    }

    # Convert the dictionary to JSON
    json_data = json.dumps(result)

    json_filename = 'prediction.json'
    json_base_path = 'json/'
    json_base_name = os.path.splitext(json_filename)[0]
    json_ext = os.path.splitext(json_filename)[1]
    new_json_name = json_base_name + json_ext
    
    # Buat increment jika file sudah ada di folder '/json'
    j = 1
    while os.path.exists(json_base_path + json_new_name):
        new_jso_name = f"{json_base_name}_{j}{json_ext}"
        j += 1
    
    json_full_path = json_base_path + json_new_name
    
    # Write the JSON data to a file
    with open(json_full_path, 'w') as json_file:
        json_file.write(json_data)

In [24]:
link = "https://cf.shopee.co.id/file/495b41bc9054f98b2aac93f4f0d19cf0"
predict(link)

1/1 [==============================] - 0s 16ms/step
Predicted Label: Capris


UnboundLocalError: local variable 'json_new_name' referenced before assignment